In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import tensorflow as tf

from src.loaders.data_loader import load_train_data, load_basic_preprocessed_train, load_test_data
from src.preprocessors.preprocessors import split_train_valid_test, do_common_preprocess, apply_minmax_scale

from src.model.multiple_output.convolution import Convolution2DVarious
from src.loaders.data_loader import load_test_features, load_basic_preprocessed_predict, load_submission_data
from src.loaders.window_generator import WindowGenerator
from src.trainers import compile_and_fit_with_pinball_loss

df = load_train_data()
df = do_common_preprocess(df)

np.random.seed(42)
tf.random.set_seed(42)

In [2]:
cutter = [
    'DHI', 
    'DNI', 
#     'WS', 
#     'RH', 
#     'T', 
#     'Hour_sin',
#     'Hour_cos', 
    'GHI', 
#     'TARGET_ROLLING_MEAN_3_shift_1',
#     'TARGET_ROLLING_MEAN_5_shift_2', 
#     'TARGET_ROLLING_MEAN_11_shift_5',
#     'TARGET_ROLLING_MEAN_23_shift_11', 
#     'TARGET_ROLLING_MEAN_47_shift_23',
    'scaled_TARGET',
    "TARGET",
]



scaler = "minmax"
days = 7

In [3]:
def preprocess(train_df, valid_df, test_df, cutter):
    train_target = train_df["TARGET"]
    valid_target = valid_df["TARGET"]
    test_target = test_df["TARGET"]
    
    train_df, valid_df, test_df = apply_minmax_scale(train, valid, test)
    
    train_df["scaled_TARGET"] = train_df["TARGET"]
    valid_df["scaled_TARGET"] = valid_df["TARGET"]
    test_df["scaled_TARGET"] = test_df["TARGET"]
    
    train_df["TARGET"] = train_target
    valid_df["TARGET"] = valid_target
    test_df["TARGET"] = test_target
    
    train_df = train_df[cutter]
    valid_df = valid_df[cutter]
    test_df = test_df[cutter]
    
    return train_df, valid_df, test_df

In [4]:
def learn(days, train, valid, test):
    submission_df = load_submission_data()
    predict = load_test_data()
    predict = do_common_preprocess(predict)
    _, predict_df, _ = apply_minmax_scale(train, predict, predict)
    
    predict_df["scaled_TARGET"] = predict_df["TARGET"]
    predict_df = predict_df[cutter]
    predict_df.drop("TARGET", axis=1, inplace=True)
    col = predict_df.columns
    predict_df = load_test_features(predict_df, 48 * days)

    train, valid, test = preprocess(train, valid, test, cutter)
    one_days_window_label_columns = WindowGenerator(
        train,
        valid,
        test,
        input_width=48 * days,
        label_width=96,
        shift=96,
        sequence_stride=1,
        label_columns=["TARGET"]
    )
    
    total_error = 0
    
    for i in range(1, 10):
        q = i/10
        print(q)
        
        conv_various = Convolution2DVarious(days, 96, train.shape[-1] - 1)
    
        compile_and_fit_with_pinball_loss(conv_various, one_days_window_label_columns, q)
        total_error += conv_various.evaluate(one_days_window_label_columns.test, verbose=0)[0]
        predict_np = predict_df.reshape(-1, days, 48, train.shape[-1] - 1)
        pred_y = conv_various.predict(predict_np)[:, :, -1]
        submission_df[f"q_{q}"] = pred_y.reshape(-1)

    return total_error, submission_df

In [5]:
df.shape[0]

52560

In [6]:
length = df.shape[0]
boundary = [
#     (slice(0, int(length * 0.6)),slice(int(length * 0.6), int(length * 0.9)), slice(int(length * 0.9), int(length * 1.0))),
#     (slice(int(length * 0.1), int(length * 0.7)),slice(int(length * 0.7), int(length * 1)), slice(int(length * 0), int(length * 0.1))),    
    (slice(int(length * 0.3), int(length * 0.9)),slice(int(length * 0.9), int(length * 1)), slice(int(length * 0), int(length * 0.3))),
    (slice(int(length * 0.4), int(length * 1)),slice(int(length * 0), int(length * 0.3)), slice(int(length * 0.3), int(length * 0.4))),
]

total_error = []
result_list = []

for train_slice, valid_slice, test_slice in boundary:
    train, valid, test = df[train_slice], df[valid_slice], df[test_slice]
    
    error, result_df = learn(days, train, valid, test)
    total_error.append(error)
    result_list.append(result_df)

0.1
Epoch 1/1000
973/973 [==============================] - 30s 20ms/step - loss: 1.7399 - mean_absolute_error: 16.5817 - mean_squared_error: 854.5178 - _pinball_loss: 1.7399 - val_loss: 1.0356 - val_mean_absolute_error: 8.1932 - val_mean_squared_error: 244.7887 - val__pinball_loss: 1.0357
Epoch 2/1000
973/973 [==============================] - 8s 8ms/step - loss: 1.5506 - mean_absolute_error: 13.5037 - mean_squared_error: 577.6739 - _pinball_loss: 1.5506 - val_loss: 1.0206 - val_mean_absolute_error: 7.7375 - val_mean_squared_error: 218.0577 - val__pinball_loss: 1.0206
Epoch 3/1000
973/973 [==============================] - 8s 8ms/step - loss: 1.5189 - mean_absolute_error: 13.0443 - mean_squared_error: 542.5162 - _pinball_loss: 1.5189 - val_loss: 1.0110 - val_mean_absolute_error: 7.5763 - val_mean_squared_error: 207.8334 - val__pinball_loss: 1.0110
Epoch 4/1000
973/973 [==============================] - 8s 8ms/step - loss: 1.5015 - mean_absolute_error: 12.7895 - mean_squared_error: 524

Epoch 13/1000
973/973 [==============================] - 7s 8ms/step - loss: 2.2621 - mean_absolute_error: 8.2534 - mean_squared_error: 237.3312 - _pinball_loss: 2.2621 - val_loss: 1.5766 - val_mean_absolute_error: 5.3483 - val_mean_squared_error: 98.9317 - val__pinball_loss: 1.5766
Epoch 14/1000
973/973 [==============================] - 7s 8ms/step - loss: 2.2674 - mean_absolute_error: 8.3435 - mean_squared_error: 243.4381 - _pinball_loss: 2.2674 - val_loss: 1.5815 - val_mean_absolute_error: 5.3131 - val_mean_squared_error: 98.3925 - val__pinball_loss: 1.5813
0.3
Epoch 1/1000
973/973 [==============================] - 8s 8ms/step - loss: 4.1007 - mean_absolute_error: 12.0230 - mean_squared_error: 492.9341 - _pinball_loss: 4.1007 - val_loss: 2.0621 - val_mean_absolute_error: 4.8101 - val_mean_squared_error: 81.2657 - val__pinball_loss: 2.0619
Epoch 2/1000
973/973 [==============================] - 7s 8ms/step - loss: 2.9335 - mean_absolute_error: 7.1917 - mean_squared_error: 171.2083 

Epoch 2/1000
973/973 [==============================] - 8s 8ms/step - loss: 2.9560 - mean_absolute_error: 5.9119 - mean_squared_error: 152.0029 - _pinball_loss: 2.9560 - val_loss: 1.9290 - val_mean_absolute_error: 3.8581 - val_mean_squared_error: 71.1426 - val__pinball_loss: 1.9289
Epoch 3/1000
973/973 [==============================] - 8s 8ms/step - loss: 2.8683 - mean_absolute_error: 5.7365 - mean_squared_error: 149.3247 - _pinball_loss: 2.8683 - val_loss: 1.8769 - val_mean_absolute_error: 3.7538 - val_mean_squared_error: 71.5960 - val__pinball_loss: 1.8772
Epoch 4/1000
973/973 [==============================] - 8s 8ms/step - loss: 2.8171 - mean_absolute_error: 5.6343 - mean_squared_error: 147.2565 - _pinball_loss: 2.8171 - val_loss: 1.8348 - val_mean_absolute_error: 3.6696 - val_mean_squared_error: 72.7422 - val__pinball_loss: 1.8350
Epoch 5/1000
973/973 [==============================] - 8s 8ms/step - loss: 2.7823 - mean_absolute_error: 5.5645 - mean_squared_error: 146.3919 - _pinb

Epoch 12/1000
973/973 [==============================] - 8s 9ms/step - loss: 1.4611 - mean_absolute_error: 6.4398 - mean_squared_error: 217.3268 - _pinball_loss: 1.4611 - val_loss: 0.9928 - val_mean_absolute_error: 4.3847 - val_mean_squared_error: 107.7520 - val__pinball_loss: 0.9927
Epoch 13/1000
973/973 [==============================] - 8s 9ms/step - loss: 1.4521 - mean_absolute_error: 6.4229 - mean_squared_error: 217.3738 - _pinball_loss: 1.4521 - val_loss: 0.9959 - val_mean_absolute_error: 4.1482 - val_mean_squared_error: 103.5123 - val__pinball_loss: 0.9958
Epoch 14/1000
973/973 [==============================] - 8s 9ms/step - loss: 1.4467 - mean_absolute_error: 6.3964 - mean_squared_error: 216.4941 - _pinball_loss: 1.4467 - val_loss: 0.9859 - val_mean_absolute_error: 4.4358 - val_mean_squared_error: 110.5483 - val__pinball_loss: 0.9859
Epoch 15/1000
973/973 [==============================] - 8s 9ms/step - loss: 1.4309 - mean_absolute_error: 6.3567 - mean_squared_error: 215.8467 

Epoch 18/1000
973/973 [==============================] - 11s 11ms/step - loss: 1.4978 - mean_absolute_error: 12.7628 - mean_squared_error: 527.8973 - _pinball_loss: 1.4978 - val_loss: 1.5016 - val_mean_absolute_error: 12.7133 - val_mean_squared_error: 519.8667 - val__pinball_loss: 1.5019
Epoch 19/1000
973/973 [==============================] - 11s 11ms/step - loss: 1.4938 - mean_absolute_error: 12.7260 - mean_squared_error: 525.3066 - _pinball_loss: 1.4938 - val_loss: 1.4983 - val_mean_absolute_error: 12.7702 - val_mean_squared_error: 523.4275 - val__pinball_loss: 1.4983
Epoch 20/1000
973/973 [==============================] - 11s 11ms/step - loss: 1.4941 - mean_absolute_error: 12.7334 - mean_squared_error: 525.5328 - _pinball_loss: 1.4941 - val_loss: 1.4960 - val_mean_absolute_error: 12.6986 - val_mean_squared_error: 517.8383 - val__pinball_loss: 1.4962
Epoch 21/1000
973/973 [==============================] - 11s 11ms/step - loss: 1.4892 - mean_absolute_error: 12.6521 - mean_squared_e

Epoch 11/1000
973/973 [==============================] - 11s 12ms/step - loss: 2.4327 - mean_absolute_error: 9.1837 - mean_squared_error: 288.1578 - _pinball_loss: 2.4327 - val_loss: 2.4867 - val_mean_absolute_error: 9.1870 - val_mean_squared_error: 285.1725 - val__pinball_loss: 2.4865
Epoch 12/1000
973/973 [==============================] - 11s 12ms/step - loss: 2.4267 - mean_absolute_error: 9.1393 - mean_squared_error: 285.6948 - _pinball_loss: 2.4267 - val_loss: 2.4774 - val_mean_absolute_error: 9.1583 - val_mean_squared_error: 284.9299 - val__pinball_loss: 2.4772
Epoch 13/1000
973/973 [==============================] - 11s 12ms/step - loss: 2.4208 - mean_absolute_error: 9.1114 - mean_squared_error: 284.7877 - _pinball_loss: 2.4208 - val_loss: 2.4712 - val_mean_absolute_error: 9.2927 - val_mean_squared_error: 290.7769 - val__pinball_loss: 2.4719
Epoch 14/1000
973/973 [==============================] - 12s 12ms/step - loss: 2.4140 - mean_absolute_error: 9.1062 - mean_squared_error: 2

973/973 [==============================] - 12s 13ms/step - loss: 2.5601 - mean_absolute_error: 5.8331 - mean_squared_error: 173.4347 - _pinball_loss: 2.5601 - val_loss: 2.8883 - val_mean_absolute_error: 6.5381 - val_mean_squared_error: 191.1805 - val__pinball_loss: 2.8871
Epoch 6/1000
973/973 [==============================] - 12s 13ms/step - loss: 2.5371 - mean_absolute_error: 5.7830 - mean_squared_error: 172.2335 - _pinball_loss: 2.5371 - val_loss: 2.8964 - val_mean_absolute_error: 6.5488 - val_mean_squared_error: 189.7570 - val__pinball_loss: 2.8967
Epoch 7/1000
973/973 [==============================] - 12s 13ms/step - loss: 2.5200 - mean_absolute_error: 5.7519 - mean_squared_error: 171.5966 - _pinball_loss: 2.5200 - val_loss: 2.8923 - val_mean_absolute_error: 6.7051 - val_mean_squared_error: 203.2884 - val__pinball_loss: 2.8908
Epoch 8/1000
973/973 [==============================] - 12s 13ms/step - loss: 2.5120 - mean_absolute_error: 5.7476 - mean_squared_error: 172.3342 - _pinbal

973/973 [==============================] - 13s 13ms/step - loss: 1.9980 - mean_absolute_error: 5.9384 - mean_squared_error: 195.9266 - _pinball_loss: 1.9980 - val_loss: 2.3784 - val_mean_absolute_error: 7.1857 - val_mean_squared_error: 237.1941 - val__pinball_loss: 2.3784
Epoch 16/1000
973/973 [==============================] - 13s 13ms/step - loss: 1.9923 - mean_absolute_error: 5.9329 - mean_squared_error: 195.6013 - _pinball_loss: 1.9923 - val_loss: 2.3376 - val_mean_absolute_error: 7.0308 - val_mean_squared_error: 236.2008 - val__pinball_loss: 2.3381
Epoch 17/1000
973/973 [==============================] - 13s 13ms/step - loss: 1.9767 - mean_absolute_error: 5.8950 - mean_squared_error: 195.1265 - _pinball_loss: 1.9767 - val_loss: 2.3634 - val_mean_absolute_error: 7.1400 - val_mean_squared_error: 241.1523 - val__pinball_loss: 2.3635
Epoch 18/1000
973/973 [==============================] - 13s 13ms/step - loss: 1.9760 - mean_absolute_error: 5.8934 - mean_squared_error: 194.7945 - _pin

973/973 [==============================] - 14s 14ms/step - loss: 1.4026 - mean_absolute_error: 6.2755 - mean_squared_error: 216.1987 - _pinball_loss: 1.4026 - val_loss: 1.7533 - val_mean_absolute_error: 7.1877 - val_mean_squared_error: 246.1401 - val__pinball_loss: 1.75373 - mean_absolute_error: 6.2757 - mean_squared_error: 215.1318 - _pinball_lo - ETA: 5s - loss: 1.4032 - mean_absolute_error: 6 - ETA: 3s - loss: 1.4018 - mean_absolute_er - ETA: 2s - loss: 1.4017 - mean
Epoch 23/1000
973/973 [==============================] - 14s 14ms/step - loss: 1.4033 - mean_absolute_error: 6.2618 - mean_squared_error: 215.1286 - _pinball_loss: 1.4033 - val_loss: 1.7409 - val_mean_absolute_error: 7.8350 - val_mean_squared_error: 275.7146 - val__pinball_loss: 1.7412ror: 6.2613 - mean_squared_error: 215.0817 - _pinball_loss: 
Epoch 24/1000
973/973 [==============================] - 14s 14ms/step - loss: 1.4012 - mean_absolute_error: 6.2847 - mean_squared_error: 216.9086 - _pinball_loss: 1.4012 - val_l

In [1]:
total_error/ 2/9

NameError: name 'total_error' is not defined

In [12]:
result_list = [
    pd.read_csv('cnn_data_split_1.csv'),
    pd.read_csv('cnn_data_split_2.csv'),
    pd.read_csv('cnn_data_split_3.csv'),
    pd.read_csv('cnn_data_split_4.csv'),
]

In [13]:
submission_df = load_submission_data()

for result_df in result_list:
    for i in range(1, 10):
        q = i/10
        submission_df[f"q_{q}"] += result_df[f"q_{q}"]
                      
for i in range(1, 10):
    q = i/10
    submission_df[f"q_{q}"] = submission_df[f"q_{q}"] / 4

In [19]:
submission_df.to_csv('cnn_data_split_2021_01_14.csv', index=False)

In [20]:
lightgbm_df = pd.read_csv('lightgbm_gbdt_split_dataset_2021_01_13.csv')

for i in range(1, 10):
    q = i/10
    submission_df[f"q_{q}"] = (submission_df[f"q_{q}"] + lightgbm_df[f"q_{q}"]) / 2

In [22]:
for i in range(1, 10):
    q = i/10
    submission_df[f"q_{q}"][submission_df[f"q_{q}"] < 0] = 0

C:\Users\bees1\anaconda3\envs\dacon\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bees1\anaconda3\envs\dacon\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bees1\anaconda3\envs\dacon\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [24]:
submission_df.to_csv('cnn_n_lgb_split_data.csv', index=False)